In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
5,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
    "conf": {
        "spark.sql.extensions" : "io.delta.sql.DeltaSparkSessionExtension",
        "spark.sql.catalog.spark_catalog": "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        "spark.jars": "/usr/share/aws/delta/lib/delta-core.jar,/usr/share/aws/delta/lib/delta-storage.jar,/usr/share/aws/delta/lib/delta-storage-s3-dynamodb.jar",
        "spark.hadoop.hive.metastore.client.factory.class": "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory"
    }
}

Starting Spark application


ID,Kind,State,Spark UI,Driver log,User,Current session?
6,pyspark,idle,Link,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,Kind,State,Spark UI,Driver log,User,Current session?
6,pyspark,idle,Link,,None,✔


In [107]:
s3_path = "s3://snowflake-ev-data/raw_data/data.json"
raw_data_df = spark.read.option("allowUnquotedFieldNames", "true").json(s3_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, IntegerType, LongType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
df = raw_data_df.withColumn(
    "data",
    F.explode(col("data").cast(ArrayType(ArrayType(StringType()))))
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
df = df.select("data")
df.head(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(data=['row-zt4k~iszy.uhv6', '00000000-0000-0000-62B4-C1BC527B773A', '0', '1676414233', None, '1676414284', None, '{ }', '5YJ3E1EA8J', 'San Diego', 'Oceanside', 'CA', '92051', '2018', 'TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)', 'Clean Alternative Fuel Vehicle Eligible', '215', '0', None, '153998050', None, None, '06073018509', None, None, None]), Row(data=['row-5r58~kb8y.789r', '00000000-0000-0000-B54E-F27AFFF902F6', '0', '1676414233', None, '1676414284', None, '{ }', '3FA6P0PU7H', 'Sedgwick', 'Derby', 'KS', '67037', '2017', 'FORD', 'FUSION', 'Plug-in Hybrid Electric Vehicle (PHEV)', 'Not eligible due to low battery range', '21', '0', None, '138214331', 'POINT (-97.27013 37.54531)', None, '20173009801', '1291', None, None]), Row(data=['row-84ix~3wif_u9ju', '00000000-0000-0000-F67B-BBFF22B88E48', '0', '1676414233', None, '1676414298', None, '{ }', '1N4AZ0CP8D', 'Snohomish', 'Marysville', 'WA', '98271', '2013', 'NISSAN', 'LEAF', 'Battery Electric Vehicle (BEV)', 'Clean Alte

In [111]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, \
    TimestampType, BooleanType, ArrayType

# Define schema elements based on your data (modify as needed)
data_item_schema = StructType([
  StructField("sid", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("id", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("position", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("created_at", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("created_meta", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("updated_at", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("updated_meta", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("meta", StringType(), True),  # Assuming hidden flag doesn't affect usage
  StructField("vin_1_10", StringType(), True),
  StructField("county", StringType(), True),
  StructField("city", StringType(), True),
  StructField("state", StringType(), True),
  StructField("zip_code", StringType(), True),
  StructField("model_year", StringType(), True),
  StructField("make", StringType(), True),
  StructField("model", StringType(), True),
  StructField("ev_type", StringType(), True),
  StructField("cafv_type", StringType(), True),
  StructField("electric_range", StringType(), True),  # Assuming numeric representation for range
  StructField("base_msrp", StringType(), True),  # Assuming numeric representation for MSRP
  StructField("legislative_district", StringType(), True),
  StructField("dol_vehicle_id", StringType(), True),
  StructField("geocoded_column", StringType(), True),  # Assuming point data for location
  StructField("electric_utility", StringType(), True),
  StructField("_2020_census_tract", StringType(), True),
  StructField(":@computed_region_x4ys_rtnd", StringType(), True),  # Computed field, data type might change
  StructField(":@computed_region_fny7_vc3j", StringType(), True),  # Computed field, data type might change
  StructField(":@computed_region_8ddd_yn5v", StringType(), True),  # Computed field, data type might change
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [112]:
# Access elements within the map function using the actual row argument
new_data = df.rdd.map(lambda row: Row(**dict(zip([f.name for f in data_item_schema.fields], 
                                                  [row.data[i] for i in range(len(data_item_schema.fields))]))))

# Create a new DataFrame with the applied schema
new_df = spark.createDataFrame(new_data, data_item_schema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [122]:
#cast the columns to correct datatypes
new_df = new_df.withColumn("electric_range", new_df["electric_range"].cast(IntegerType())).withColumn("base_msrp", new_df["base_msrp"].cast(IntegerType())).withColumn(":@computed_region_x4ys_rtnd", new_df[":@computed_region_x4ys_rtnd"].cast(IntegerType())).withColumn(":@computed_region_fny7_vc3j", new_df[":@computed_region_fny7_vc3j"].cast(IntegerType())).withColumn(":@computed_region_8ddd_yn5v", new_df[":@computed_region_8ddd_yn5v"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [126]:
def check_for_nulls(df, column_list):
    """Check for null values in specified columns."""
    for column in column_list:
        null_count = df.filter(col(column).isNull()).count()
        if null_count > 0:
            print(f"Warning: Found {null_count} null values in column '{column}'.")
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [127]:
#check for null values in the columns
new_df = check_for_nulls(new_df, ["electric_range", "base_msrp", ":@computed_region_x4ys_rtnd", ":@computed_region_fny7_vc3j", ":@computed_region_8ddd_yn5v"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
def check_for_duplicates(df, subset):
    """Check for duplicate rows based on a subset of columns."""
    initial_count = df.count()
    dedup_count = df.dropDuplicates(subset).count()
    if initial_count != dedup_count:
        print(f"Warning: Found {initial_count - dedup_count} duplicate rows based on columns {subset}.")
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [129]:
#check for duplicates
new_df = check_for_duplicates(new_df, ["sid"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
def check_value_ranges(df, column, min_val, max_val):
    """Check that values in a specific column fall within a specified range."""
    out_of_range_count = df.filter((col(column) < min_val) | (col(column) > max_val)).count()
    if out_of_range_count > 0:
        print(f"Warning: {out_of_range_count} out-of-range values found in column '{column}'.")
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
#check value ranges
new_df = check_value_ranges(new_df, "electric_range", 0, 1000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
#save the dataframe to s3bucket in parquet format
new_df.write.mode("overwrite").parquet("s3://snowflake-ev-data/processed_data/data.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…